# Лабораторная работа №6
### "ИНФОРМАЦИОННОЕ ПРОТИВОБОРСТВО"

**Вариант:** 1

**Цель работы:** изучить теоретико-игровую модель информационного противоборства в социальных сетях. Промоделировать информационное управление в рамках игры и определить итоговое мнение агентов.

In [15]:
import logging

import numpy as np

np.set_printoptions(precision=2, suppress=True)
logging.basicConfig(level=logging.INFO, format='%(message)s')

In [16]:
# Размерность матрицы (количество агентов)
N = 10

### 1. Генерация стохастической матрицы
В модели социальной сети рассматриваются агенты ($i \in N = {1,2,..., n}, n \in \mathbb{N}$), составляющие эту сеть, и их
мнения $х_i$, зависящие от дискретного момента времени: $x_i = x_i(t), t = 0, 1, 2, ...$. 

В каждый момент времени мнения агентов изменяются под влиянием мнений других агентов. Степень этого влияния определяется неотрицательной стохастической по строкам матрицей $А = (a_{ij})$, где $a_{ij}$ — степень доверия $і$-го агента $j$-му агенту (в том числе и самому себе).

In [17]:
def generate_trust_matrix(n_agents):
    matrix = np.random.rand(n_agents, n_agents)
    # Нормализуем строки и столбцы, чтобы сумма значений в каждой строке была равна 1.
    return matrix / matrix.sum(axis=1, keepdims=True)


# Функция для проверки стохастичности матрицы
def is_stochastic(matrix):
    rows_sum = np.sum(matrix, axis=1)
    logging.info(f"Сумма по строкам: {rows_sum}")
    # Проверяем, что сумма значений в строках равна 1.
    return np.allclose(rows_sum, 1)


# Генерируем стохастическую матрицу доверия
trust_matrix = generate_trust_matrix(N)
trust_matrix

array([[0.18, 0.08, 0.02, 0.05, 0.07, 0.17, 0.08, 0.03, 0.16, 0.15],
       [0.1 , 0.13, 0.04, 0.13, 0.06, 0.15, 0.08, 0.08, 0.08, 0.15],
       [0.03, 0.17, 0.11, 0.02, 0.13, 0.1 , 0.1 , 0.14, 0.15, 0.05],
       [0.09, 0.17, 0.11, 0.14, 0.04, 0.11, 0.09, 0.03, 0.13, 0.09],
       [0.09, 0.1 , 0.02, 0.08, 0.09, 0.12, 0.13, 0.14, 0.01, 0.22],
       [0.13, 0.  , 0.05, 0.01, 0.13, 0.14, 0.14, 0.14, 0.13, 0.14],
       [0.07, 0.07, 0.01, 0.2 , 0.08, 0.14, 0.04, 0.01, 0.2 , 0.16],
       [0.12, 0.16, 0.18, 0.04, 0.13, 0.11, 0.04, 0.09, 0.08, 0.05],
       [0.15, 0.11, 0.06, 0.09, 0.13, 0.12, 0.11, 0.01, 0.12, 0.11],
       [0.05, 0.11, 0.09, 0.11, 0.1 , 0.08, 0.16, 0.16, 0.05, 0.11]])

In [18]:
is_stochastic(trust_matrix)

Сумма по строкам: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


True

### 2. Мнение агентов

Вектор мнений $X = (x_1, x_2, ..., x_n)$ задан для начального момента времени ($X(0)$) и изменяется следующим образом:
$$x_i(t) = \sum_{j = 0}^{n} a_{ij}x_{j}(t - 1)$$

Этот закон можно записать иначе:
$$X(t) = A \cdot X(t ー 1)$$

При достаточно долгом взаимодействии агентов вектор мнений сходится к итоговому значению:
$$X = A^{\infty} \cdot X(0)$$
, где $A^{\infty} = \lim_{t \to \infty} A^{t}$.

In [19]:
def compute_final_opinions(trust_matrix, initial_opinions, epsilon=1e-6):
    i = 0
    logging.info(f"x({i}) = {initial_opinions}")
    current_opinions = initial_opinions.copy()
    previous_opinions = np.zeros_like(initial_opinions)
    res_trust_matrix = trust_matrix.copy()
    while np.linalg.norm(current_opinions - previous_opinions) > epsilon:
        previous_opinions = current_opinions
        current_opinions = np.dot(trust_matrix, current_opinions)
        res_trust_matrix = np.dot(res_trust_matrix, trust_matrix)
        i += 1
        logging.info(f"x({i}) = {current_opinions}")

    return current_opinions, res_trust_matrix

In [20]:
a, b = 1, 20
# Генерируем начальные мнения агентов.
initial_opinions = np.random.randint(a, b + 1, N)
final_opinions, res_trust_matrix = compute_final_opinions(trust_matrix, initial_opinions)
print(f"После взаимодействия агентов, вектор мнений сходится к значению:\nX = {final_opinions}")
print(f"Результирующая матрица доверия:\n{res_trust_matrix}")

x(0) = [ 9  4 13  4  2  1  6  6 20 10]
x(1) = [8.21 6.77 7.72 7.92 6.13 7.84 8.29 7.48 7.39 6.65]
x(2) = [7.47 7.44 7.29 7.48 7.36 7.44 7.39 7.36 7.43 7.43]
x(3) = [7.43 7.43 7.4  7.42 7.42 7.41 7.44 7.4  7.42 7.4 ]
x(4) = [7.42 7.42 7.41 7.42 7.41 7.41 7.42 7.41 7.42 7.41]
x(5) = [7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42]
x(6) = [7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42]
x(7) = [7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42]
x(8) = [7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42]
x(9) = [7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42]
x(10) = [7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42]


После взаимодействия агентов, вектор мнений сходится к значению:
X = [7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42 7.42]
Результирующая матрица доверия:
[[0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]]


### 3. Информационное управление
Для информационного управления необходимо наличие нескольких игроков (в данном случае - двух), которые могут влиять на начальные мнения некоторых агентов. Случайным образом выберем число и номера агентов первого игрока и аналогично число и номера агентов второго игрока (они не должны пересекаться, так как эта ситуация выходит за рамки данной работы). Следует обратить внимание, что число агентов влияния первого игрока не обязательно должно совпадать с числом агентов влияния второго игрока.

In [21]:
def get_player_agents_indices(n_agents):
    # Генерируем случайное число агентов для каждого игрока (необязательно равное).
    player1_count = np.random.randint(1, n_agents)
    # Учитываем уже выбранных агентов для первого игрока.
    player2_count = np.random.randint(1, n_agents - player1_count + 1)  

    # Генерируем случайные неповторяющиеся индексы агентов для каждого игрока
    player1_indices = np.random.choice(
        range(n_agents), 
        size=player1_count, replace=False)
    player2_indices = np.random.choice(
        [idx for idx in range(n_agents) if idx not in player1_indices], 
        size=player2_count, replace=False)
    
    player1_indices.sort()
    player2_indices.sort()
    return player1_indices, player2_indices

In [22]:
player1_influence_indices, player2_influence_indices = get_player_agents_indices(N)
print(f"Индексы агентов влияния для первого игрока: {player1_influence_indices}")
print(f"Индексы агентов влияния для второго игрока: {player2_influence_indices}")

Индексы агентов влияния для первого игрока: [0 1 3 4 5 7 9]
Индексы агентов влияния для второго игрока: [2 8]


Затем сгенерируем случайным образом управление для агентов влияния первого и второго игроков.
Сформируем вектор начальных мнений агентов с учетом информационного влияния, нейтральным агентам назначим случайные стартовые мнения. 

In [23]:
ABS_MAX_INFLUENCE = 100
player1_influence = np.random.randint(0, ABS_MAX_INFLUENCE)
print(f"Сформированное начальное мнение агентов первого игрока: {player1_influence}")
player2_influence = np.random.randint(-ABS_MAX_INFLUENCE, 0)
print(f"Сформированное начальное мнение агентов второго игрока: {player2_influence}")

Сформированное начальное мнение агентов первого игрока: 54
Сформированное начальное мнение агентов второго игрока: -72


In [24]:
# Модифицируем начальные мнения агентов с учётом влияния.
for i in player1_influence_indices:
    initial_opinions[i] = player1_influence
    
for i in player2_influence_indices:
    initial_opinions[i] = player2_influence

final_opinions, res_trust_matrix = compute_final_opinions(trust_matrix, initial_opinions)
print(f"После взаимодействия агентов, вектор мнений сходится к значению:\nX = {final_opinions}")
print(f"Результирующая матрица доверия:\n{res_trust_matrix}")

x(0) = [ 54  54 -72  54  54  54   6  54 -72  54]
x(1) = [27.27 34.82 16.99 19.4  43.83 24.86 24.7  19.32 26.94 29.16]
x(2) = [27.79 26.85 27.92 26.4  27.33 27.29 27.06 27.4  28.37 26.4 ]
x(3) = [27.34 27.13 27.39 27.25 27.03 27.37 27.19 27.4  27.28 27.17]
x(4) = [27.26 27.25 27.25 27.25 27.24 27.26 27.24 27.26 27.24 27.25]
x(5) = [27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25]
x(6) = [27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25]
x(7) = [27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25]
x(8) = [27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25]
x(9) = [27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25]
x(10) = [27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25]
x(11) = [27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25]


После взаимодействия агентов, вектор мнений сходится к значению:
X = [27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25 27.25]
Результирующая матрица доверия:
[[0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]
 [0.1  0.1  0.07 0.09 0.09 0.12 0.1  0.08 0.11 0.13]]


In [25]:
if np.isclose(final_opinions[0], 0):
    print("Ничья")
elif final_opinions[0] > 0:
    print("Выиграл первый игрок")
else:
    print("Выиграл второй игрок") 

Выиграл первый игрок
